In [7]:
# pip install dask

In [8]:
# pip install dask[distributed]

In [9]:
# pip install dask[graphviz]

In [46]:
import dask
from dask.distributed import Client, LocalCluster
import os
import dask.dataframe as dd
import pandas as pd
import numpy as np
import time

In [39]:
from concurrent.futures import thread
client = Client(
    n_workers=4,
    threads_per_worker=1,
    memory_limit='4GB'
  )
client

/usr/local/lib/python3.10/dist-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39529 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:43099
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:39529/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35243'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37305'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:39807'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45169'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:35933', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:35933
INFO:distributed.core:Starting established conne

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:39529/status,
Dashboard: http://127.0.0.1:39529/status,Workers: 4
Total threads: 4,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43099,Workers: 4
Dashboard: http://127.0.0.1:39529/status,Total threads: 4
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:38369,Total threads: 1
Dashboard: http://127.0.0.1:42361/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:35243,


In [13]:
# ddf = dd.read_csv( "zamowienia_expanded.csv", sep=';')

In [19]:
ddf=dd.read_csv("zamowienia.csv", sep=';')

In [20]:
ddf

,Kraj,Sprzedawca,Data zamowienia,idZamowienia,Utarg
npartitions=1,,,,,
,string,string,string,int64,float64
,...,...,...,...,...


In [21]:
ddf.head()

,Kraj,Sprzedawca,Data zamowienia,idZamowienia,Utarg
0,Polska,Kowalski,2003-07-16,10248,440.00
1,Polska,Sowiński,2003-07-10,10249,1863.40
2,Niemcy,Peacock,2003-07-12,10250,1552.60
3,Niemcy,Leverling,2003-07-15,10251,654.06
4,Niemcy,Peacock,2003-07-11,10252,3597.90


In [23]:
df=pd.read_csv('zamowienia.csv', sep=';')

In [25]:
for _ in range(5):
    row = np.random.randint(10, len(df))
    col = np.random.choice(df.columns)
    df.loc[row, col] = np.nan

In [26]:
# df.to_csv('zamowienia_missing.csv', index=False)

In [58]:
ddf2= dd.read_csv('zamowienia_missing.csv')

In [32]:
ddf2.head()

,Kraj,Sprzedawca,Data zamowienia,idZamowienia,Utarg
0,Polska,Kowalski,2003-07-16,10248,440.00
1,Polska,Sowiński,2003-07-10,10249,1863.40
2,Niemcy,Peacock,2003-07-12,10250,1552.60
3,Niemcy,Leverling,2003-07-15,10251,654.06
4,Niemcy,Peacock,2003-07-11,10252,3597.90


In [34]:
print(ddf2.dtypes)

Kraj               string[pyarrow]
Sprzedawca         string[pyarrow]
Data zamowienia    string[pyarrow]
idZamowienia                 int64
Utarg                      float64
dtype: object


In [35]:
print(df.dtypes)

Kraj                object
Sprzedawca          object
Data zamowienia     object
idZamowienia         int64
Utarg              float64
dtype: object


typy danych w dataframie dask zostały zmienione

In [36]:
# sum = ddf2.sum().compute()


TypeError: 'ArrowStringArray' with dtype string does not support reduction 'sum' with pyarrow version 17.0.0. 'sum' may be supported by upgrading pyarrow.

pojawia się błąd

In [59]:
top_sellers=(
    ddf2.groupby('Sprzedawca')['Utarg']
    .sum()
    .nlargest(10)
    .compute()
)

In [60]:
top_sellers

,Utarg
Sprzedawca,
Peacock,225763.68
Leverling,200727.82
Davolio,179982.09
Fuller,162503.78
Callahan,123032.67
King,112257.49
Dudek,75048.04
Sowiński,72527.63
Kowalski,68792.25


In [61]:
start_time = time.time()
df_filtered= ddf2[
    (ddf2["Data zamowienia"] >= "2019-01-01") &
    (ddf2["Data zamowienia"] <= "2019-06-30")
]

In [62]:
result = df_filtered.compute()

In [63]:
result.head()

,Kraj,Sprzedawca,Data zamowienia,idZamowienia,Utarg
